In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from utils import load_config
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

In [2]:
CURRENT=os.getcwd()
ROOT=os.path.dirname(CURRENT)

In [3]:
config_f=load_config("config.yaml")

In [4]:
datos=pd.read_csv(os.path.join(ROOT,config_f["data"]["main"]))
genres=pd.read_csv(os.path.join(ROOT,config_f["data"]["cat_genres"]))
themes=pd.read_csv(os.path.join(ROOT,config_f["data"]["cat_themes"]))

### Contexto general del proyecto:

Hemos recolectado datos de la pagina [My anime list](https://myanimelist.net) <br>
My anime list es una página que recopila información sobre todos los animes emitidos y en emisión, todas las personas interesadas pueden crear una cuenta y  colaborar en esta página para calificar los animes que han visto. <br>
En My anime list podemos encontrar la siguiente información para cada anime: <br>

+ Título del anime.
+ Calificación promedio asignada por todos los usuarios que han contribuido.
+ Ranking del anime con respecto a las calificaciones obtenidas por los otros animes.
+ Popularidad del anime (número de personas que registraron haber visto ese anime).
+ Estudio de animación.
+ Tema del anime.
+ Demografía a la que pertenece el anime.
+ Fecha de emisión
+ Número de capítulos en el periodo de emisión. 

In [5]:
number_anime=datos.shape[0]
average_score=datos["score"].mean()
number_episodes=datos["number_episodes"].sum()
number_studios=datos["studio"].nunique()

data = {"Número de animes:": [number_anime],
        "Score promedio:": [average_score],
        "Número de episodios:": [number_episodes],
        "Número de estudios:": [number_studios]}

In [6]:
df_stats = pd.DataFrame(data)

### Algunas estadísticas:  

In [7]:
# Redondea los decimales para el promedio
df_stats['Score promedio:'] = df_stats['Score promedio:'].round(2)

# Añade comas a las cifras en miles
for col in ['Número de animes:', 'Número de episodios:', 'Número de estudios:']:
    df_stats[col] = df_stats[col].apply(lambda x: "{:,}".format(x))

In [8]:
fig = go.Figure(data=[go.Table(
    header=dict(
        values=['<b>{}</b>'.format(i) for i in df_stats.columns],  # Use <b> para texto en negrita
        line_color='darkslategray',
        fill_color='lightskyblue',
        align='center',
        font=dict(color='black', size=16)  # Ajusta el tamaño y el color de la fuente
    ),
    cells=dict(
        values=[df_stats[col] for col in df_stats.columns],
        line_color='darkslategray',
        fill_color=['paleturquoise', 'white'],  # Alternar colores de relleno de celdas
        align='center',
        font=dict(color='darkslategray', size=15)  # Ajusta el tamaño y el color de la fuente
    ))
])


fig.update_layout(
    autosize=False,
    width=500,
    height=80,
    margin=dict(
        l=0,  # left margin
        r=0,  # right margin
        b=0, # bottom margin
        t=0, # top margin
        pad=100 # padding
    )
)


fig.show()


<br>
<br>
<br>


In [9]:
genres=genres\
    .sort_values(by="count",ascending=False)\
    .rename(columns={"genres": "Género","count":"Animes"})\
    .drop(columns='id')\
    .assign(
        Género = lambda df_: df_.Género.str.replace("_"," ", regex=True),
        Animes = lambda df_: df_.Animes.apply(lambda x:  "{:,}".format(x))
        )

In [10]:
fig = go.Figure(data=[go.Table(
    header=dict(
        values=['<b>{}</b>'.format(i) for i in genres.columns],  # Use <b> para texto en negrita
        line_color='darkslategray',
        fill_color='lightskyblue',
        align='center',
        font=dict(color='black', size=16)  # Ajusta el tamaño y el color de la fuente
    ),
    cells=dict(
        values=[genres[col] for col in genres.columns],
        line_color='darkslategray',
        fill=dict(color=['paleturquoise', 'white']),  # Alternar colores de relleno de celdas
        align='center',
        font=dict(color='darkslategray', size=15)  # Ajusta el tamaño y el color de la fuente
    ))
])

fig.update_layout(
    autosize=False,
    #width=500,
    height=400,
    margin=dict(
        l=0,  # left margin
        r=0,  # right margin
        b=0, # bottom margin
        t=0, # top margin
        pad=5 # padding
    )
)

fig.show()

<br>

<p float="left">
  <img src="images/gokushufu.jpeg" width="45%" height="250" />
  <img src="images/hells.jpe" width="45%" height="250"/> 
</p>


<p float="center">
  <img src="images/mushoku.jpg" width="45%" height="250" />
  <img src="images/opi.jpe" width="45%" height="250" />

</p>


<br>
<br>
El año en el que se produjeron más animes fue en el 2016. <br>
Durante la pandemia por Covid-19 muchos estudios de animación tuvieron que parar su producción, por lo que hubo una reducción en el número de animes producidos entre 2019 y 2020.


<br>
<br>
<br>


In [11]:
# Primero, calcula el promedio de los puntajes para cada año
avg_scores_per_year = datos.groupby('year')['score'].mean().reset_index()

fig = px.histogram(datos[datos['year'] >= 1960], x="year",
                   title='Distribución de los animes en el tiempo',
                   labels={'year':'Año'},
                   opacity=0.8,
                   log_y=False,
                   color_discrete_sequence=['lime'],
                   nbins=len(datos[datos['year'] >= 1960]['year'].unique())
                   )

fig.update_layout(
    title=dict(text="<b>Distribución de los animes en el tiempo</b>", font=dict(color='white')),
    xaxis=dict(
        title="<b>Año</b>",
        titlefont=dict(color='white'),
        tickfont=dict(color='white'),
        linecolor='white',
        showgrid=False
    ),
    yaxis=dict(
        title="<b>Conteo</b>",
        titlefont=dict(color='white'),
        tickfont=dict(color='white'),
        linecolor='white',
        showgrid=False
    ),
    yaxis2=dict(
        title="<b>Score promedio</b>",
        titlefont=dict(color='white'),
        tickfont=dict(color='white'),
        linecolor='white',
        overlaying='y',
        side='right',
        range=[0, 10]  
    ),
    autosize=False,
    width=800,
    height=500,
    plot_bgcolor='rgba(0,0,0,0)',  
    paper_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=50, r=50, b=100, t=100, pad=5)
)

fig.add_trace(
    go.Scatter(
        x=avg_scores_per_year[avg_scores_per_year['year'] >= 1960]['year'],
        y=avg_scores_per_year[avg_scores_per_year['year'] >= 1960]['score'],
        mode='lines+markers',
        marker_color='white',
        name='Score Promedio',
        yaxis='y2',
        line=dict(color='white')
    )
)

# Actualizamos los límites del eje x
fig.update_xaxes(range=[1960, datos['year'].max()])

fig.show()



## Poblaciones objetivo

<br>
<br>

### Shounen

| Descripción | Ejemplo |
|-------------|--------|
| Dirigido principalmente a adolescentes varones, presenta historias de acción, aventura y amistad. <br> Ejemplos:<br>  - Naruto <br> - My Hero Academia <br>- Attack on Titan <br>- Full Metal Alchemist<br> - Bleach | ![Shounen](images/naruto.jpg) |

### Seinen

| Descripción | Ejemplo |
|-------------|--------|
| Dirigido a jóvenes adultos varones, suele abordar temas más maduros y complejos, con narrativas más serias y realistas. <br> Ejemplos:<br>  - Psyco-Pass <br> - Kaguya-Sama <br>- Oshi no ko <br>- Monster <br> - Vindland Saga | ![Seinen](images/pysco.jpg) |

### Shoujo

| Descripción | Ejemplo |
|-------------|--------|
| Dirigido a adolescentes y jóvenes adultas mujeres, se enfoca en historias románticas y emocionales, con énfasis en las relaciones y el crecimiento personal.<br> Ejemplos:<br>  - Oremonogatari <br> - Fruits Basket <br>- Kimi ni todoke <br>- Orange <br> - Banana Fish | ![Shoujo](images/ore.jpeg) |

### Josei


| Descripción | Ejemplo |
|-------------|--------|
| Dirigido a mujeres adultas, aborda temas más maduros y realistas en relación con la vida cotidiana, el trabajo y las relaciones. <br> Ejemplos:<br>  - Yuri on Ice <br> - Nana <br>- Usagi drop <br>- Chihayafuru <br> - Hachimitsu to Clover  | ![Josei](images/yuri.jpe) |

### Kids


| Descripción | Ejemplo |
|-------------|--------|
| Dirigido a niños de corta edad, presenta historias simples y educativas con un enfoque en el entretenimiento y la diversión. <br> Ejemplos:<br>  - Sailor Moon <br> - Digimon <br>- Doraemon <br>- Pokemon <br> - Sakura Card Captor  | ![Kids](images/sailor.jpg) |

